# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [2]:
import pandas as pd
import numpy as np
import math
import json
from pandas.io.json import json_normalize
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [3]:
pd.set_option('display.max_rows',500)
print('portfolio shape: ',portfolio.shape)
print('profile shape: ',profile.shape)
print('transcript shape: ',transcript.shape)

portfolio shape:  (10, 6)
profile shape:  (17000, 5)
transcript shape:  (306534, 4)


1) viewed offer didn't completed

2) viewed offer completed

3) Didn't view offer completed

4) Didn't get offer completed

Need to classify transactions as either 2, 3, or 4

Once the transactions are classified merge on demographic and promotion details

demographics for 1 vs 2 show likelihood of using offer
2 vs 3 shows whether offer worth while

Look at demographic clusters for 3 to get an indea of our usual customers
compare this with 2 to see if there are other demographics that the offers bring in
compare with group 1 to see if it put any group off

logistic Regression on portfolio fields and individual to get probability of using the offer (1 vs 2)

10 offer types

In [41]:
portfolio

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


In [106]:
#getting unique channels used
channels = set()
for row in portfolio['channels']:
    channels.update(set(row))

#Creating dummy variables for channels
for channel in channels:
    portfolio_transformed[channel] = 0
    
row_index = 0
for row in portfolio_transformed['channels']:
    col_index = 6
    for channel in channels:
        if channel in set(row):
            portfolio_transformed.iloc[row_index,col_index] = 1
        col_index += 1
    row_index += 1
#Creating dummy variables for offer_type
offer_dummies = pd.get_dummies(portfolio_transformed['offer_type'])
portfolio_transformed = pd.concat([portfolio_transformed, offer_dummies], axis=1)
portfolio_transformed

Channels used:  {'email', 'web', 'social', 'mobile'}


,channels,difficulty,duration,id,offer_type,reward,email,web,social,mobile,bogo,discount,informational
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,1,1,0,1,1,1,0,0
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,1,1,1,1,1,1,0,0
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,1,1,1,0,1,0,0,1
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,1,1,1,0,1,1,0,0
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,1,1,1,0,0,0,1,0
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,1,1,1,1,1,0,1,0
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,1,1,1,1,1,0,1,0
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,1,1,0,1,1,0,0,1
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,1,1,1,1,1,1,0,0
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,1,1,1,0,1,0,1,0


In [107]:
#dropping columns that are no longer needed
#Also dropping informational to reduce multicollinearity
portfolio_transformed.drop(['channels','offer_type','informational'],axis=1,inplace=True)
portfolio_transformed

,difficulty,duration,id,reward,email,web,social,mobile,bogo,discount
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,1,0,1,1,1,0
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,1,1,0
2,0,4,3f207df678b143eea3cee63160fa8bed,1,1,1,0,1,0,0
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,0,1,1,0
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,1,1,0,0,0,1
5,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1,1,0,1
6,10,10,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1,1,0,1
7,0,3,5a8bc65990b245e5a138643cd4eb9837,1,1,0,1,1,0,0
8,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1,1,1,0
9,10,7,2906b810c7d4411798c6938adc9daaa5,1,1,1,0,1,0,1


In [4]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [5]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [4]:
event_counts = transcript.event.value_counts()
event_counts

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [5]:
value_splits = json_normalize(transcript['value'])
value_splits.head()

,amount,offer id,offer_id,reward
0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,NaN,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


In [6]:
all_events_df = pd.concat([transcript, value_splits], axis=1)
all_events_df.head()

,event,person,time,value,amount,offer id,offer_id,reward
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},NaN,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


In [7]:
cols_to_check = ['amount','offer id','offer_id','reward']

nulls = all_events_df[cols_to_check].isnull()
nulls_by_event = pd.concat([all_events_df['event'], nulls], axis=1)
nulls_by_event = nulls_by_event.groupby('event').sum()
nulls_by_event = pd.concat([nulls_by_event, event_counts], axis=1, sort=True)

for col in cols_to_check:
    nulls_by_event[col] = nulls_by_event[col]/nulls_by_event['event']

nulls_by_event

,amount,offer id,offer_id,reward,event
offer completed,1.0,1.0,0.0,0.0,33579
offer received,1.0,0.0,1.0,1.0,76277
offer viewed,1.0,0.0,1.0,1.0,57725
transaction,0.0,1.0,1.0,1.0,138953


The above tables shows:
- Transactions only have amount values
- Offers viewed only have 'offer id' values
- offers recevied only have 'offer id' values
- Offers completed have both 'offer_id' and 'reward'
- There are no missing values for the fields connected to each event

### Splitting out the events into 4 distinct tables

In [8]:
#Transaction table
transaction_df = all_events_df[all_events_df['event'] == 'transaction'].copy()
transaction_df = transaction_df[['person','time','amount']].reset_index(drop=True)
transaction_df['tran_id'] = (transaction_df['person'] 
                             + "_" + transaction_df['time'].map(str) 
                             + "_" + transaction_df['amount'].round(2).map(str)
                            )
transaction_df = transaction_df[['tran_id','person','time','amount']]
print('Transaction Table')
transaction_df.head()

Transaction Table


,tran_id,person,time,amount
0,02c083884c7d45b39cc68e1314fec56c_0_0.83,02c083884c7d45b39cc68e1314fec56c,0,0.83
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f_0_34.56,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56
2,54890f68699049c2a04d415abc25e717_0_13.23,54890f68699049c2a04d415abc25e717,0,13.23
3,b2f1cd155b864803ad8334cdf13c4bd2_0_19.51,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51
4,fe97aa22dd3e48c8b143116a8403dd52_0_18.97,fe97aa22dd3e48c8b143116a8403dd52,0,18.97


In [44]:
#received table
received_df = all_events_df[all_events_df['event'] == 'offer received'].copy()
received_df = received_df[['person','time','offer id']].rename(columns={"offer id": "offer_id"}).reset_index(drop=True)
#merging on duration to received table
received_df = received_df.merge(portfolio[['id','duration']],how='left', left_on='offer_id', right_on='id')
received_df.drop('id',axis=1,inplace=True)

received_df['marketing_id'] = (received_df['person'] 
                             + "_" + received_df['time'].map(str)
                             + "_" + received_df['offer_id']
                            )
received_df.head()

,person,time,offer_id,duration,marketing_id
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,7,78afa995795e4d85b5d9ceeca43f5fef_0_9b98b8c7a33...
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,10,a03223e636434f42ac4c3df47e8bac43_0_0b1e1539f2c...
2,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5,7,e2127556f4f64592b11af22de27a7932_0_2906b810c7d...
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4,10,8ec6ce2a7e7949b1bf142def7d0e0586_0_fafdcd668e3...
4,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0,5,68617ca6246f4fbc85e91a2a49552598_0_4d5c57ea9a6...


In [11]:
#appending marker to transaction table to say if it was part in scope for a promotion
transaction_received_df = pd.merge(transaction_df[['tran_id','person','time']]
                            ,received_df
                           ,on='person'
                           ,how='inner')
transaction_received_df['In Scope'] = ((transaction_received_df['time_y'] >= transaction_received_df['time_x'])
                                     & (transaction_received_df['time_y'] <= (transaction_received_df['time_x'] 
                                                                              + transaction_received_df['duration']
                                                                             )
                                       )
                                      )
transaction_received_df.head()
#validate='m:1'

,tran_id,person,time_x,time_y,offer_id,duration,marketing_id,In Scope
0,02c083884c7d45b39cc68e1314fec56c_0_0.83,02c083884c7d45b39cc68e1314fec56c,0,0,ae264e3637204a6fb9bb56bc8210ddfd,7,02c083884c7d45b39cc68e1314fec56c_0_ae264e36372...,True
1,02c083884c7d45b39cc68e1314fec56c_0_0.83,02c083884c7d45b39cc68e1314fec56c,0,408,0b1e1539f2cc45b7b9fa7c272da2e1d7,10,02c083884c7d45b39cc68e1314fec56c_408_0b1e1539f...,False
2,02c083884c7d45b39cc68e1314fec56c_6_1.44,02c083884c7d45b39cc68e1314fec56c,6,0,ae264e3637204a6fb9bb56bc8210ddfd,7,02c083884c7d45b39cc68e1314fec56c_0_ae264e36372...,False
3,02c083884c7d45b39cc68e1314fec56c_6_1.44,02c083884c7d45b39cc68e1314fec56c,6,408,0b1e1539f2cc45b7b9fa7c272da2e1d7,10,02c083884c7d45b39cc68e1314fec56c_408_0b1e1539f...,False
4,02c083884c7d45b39cc68e1314fec56c_12_4.56,02c083884c7d45b39cc68e1314fec56c,12,0,ae264e3637204a6fb9bb56bc8210ddfd,7,02c083884c7d45b39cc68e1314fec56c_0_ae264e36372...,False


In [12]:
#appending whether transaction was eligbale to be part of a promotion to transaction table
eligable_trans = transaction_received_df[transaction_received_df['In Scope']]['tran_id'].unique()
transaction_df['eligable_promotion'] = transaction_df['tran_id'].isin(eligable_trans)
transaction_df.head()

,tran_id,person,time,amount,eligable_promotion
0,02c083884c7d45b39cc68e1314fec56c_0_0.83,02c083884c7d45b39cc68e1314fec56c,0,0.83,True
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f_0_34.56,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,True
2,54890f68699049c2a04d415abc25e717_0_13.23,54890f68699049c2a04d415abc25e717,0,13.23,False
3,b2f1cd155b864803ad8334cdf13c4bd2_0_19.51,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51,True
4,fe97aa22dd3e48c8b143116a8403dd52_0_18.97,fe97aa22dd3e48c8b143116a8403dd52,0,18.97,True


In [13]:
#viewed table
viewed_df = all_events_df[all_events_df['event'] == 'offer viewed'].copy()
viewed_df = viewed_df[['person','time','offer id']].rename(columns={"offer id": "offer_id"}).reset_index(drop=True)
viewed_df.head()

,person,time,offer_id
0,389bc3fa690240e798340f5a15918d5c,0,f19421c1d4aa40978ebb69ca19b0e20d
1,d1ede868e29245ea91818a903fec04c6,0,5a8bc65990b245e5a138643cd4eb9837
2,102e9454054946fda62242d2e176fdce,0,4d5c57ea9a6940dd891ad53e9dbe8da0
3,02c083884c7d45b39cc68e1314fec56c,0,ae264e3637204a6fb9bb56bc8210ddfd
4,be8a5d1981a2458d90b255ddc7e0d174,0,5a8bc65990b245e5a138643cd4eb9837


In [14]:
received_viewed_df = pd.merge(received_df[['marketing_id','person','time','duration','offer_id']]
                            ,viewed_df
                           ,on=['person','offer_id']
                           ,how='inner')
received_viewed_df['In Scope'] = ((received_viewed_df['time_y'] >= received_viewed_df['time_x'])
                                     & (received_viewed_df['time_y'] <= (received_viewed_df['time_x'] 
                                                                              + received_viewed_df['duration']
                                                                             )
                                       )
                                  )
received_viewed_df.head()

,marketing_id,person,time_x,duration,offer_id,time_y,In Scope
0,78afa995795e4d85b5d9ceeca43f5fef_0_9b98b8c7a33...,78afa995795e4d85b5d9ceeca43f5fef,0,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,True
1,a03223e636434f42ac4c3df47e8bac43_0_0b1e1539f2c...,a03223e636434f42ac4c3df47e8bac43,0,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,6,True
2,a03223e636434f42ac4c3df47e8bac43_0_0b1e1539f2c...,a03223e636434f42ac4c3df47e8bac43,0,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,624,False
3,a03223e636434f42ac4c3df47e8bac43_504_0b1e1539f...,a03223e636434f42ac4c3df47e8bac43,504,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,6,False
4,a03223e636434f42ac4c3df47e8bac43_504_0b1e1539f...,a03223e636434f42ac4c3df47e8bac43,504,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,624,False


In [45]:
viewed_in_time = received_viewed_df[received_viewed_df['In Scope']]['marketing_id'].unique()
received_df['viewed_in_time'] = received_df['marketing_id'].isin(viewed_in_time)
received_df.head()

,person,time,offer_id,duration,marketing_id,viewed_in_time
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,7,78afa995795e4d85b5d9ceeca43f5fef_0_9b98b8c7a33...,True
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,10,a03223e636434f42ac4c3df47e8bac43_0_0b1e1539f2c...,True
2,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5,7,e2127556f4f64592b11af22de27a7932_0_2906b810c7d...,False
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4,10,8ec6ce2a7e7949b1bf142def7d0e0586_0_fafdcd668e3...,False
4,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0,5,68617ca6246f4fbc85e91a2a49552598_0_4d5c57ea9a6...,False


In [57]:
received_df['viewed_in_time'].value_counts()

False    58606
True     17671
Name: viewed_in_time, dtype: int64

In [16]:
#completed table
completed_df = all_events_df[all_events_df['event'] == 'offer completed'].copy()
completed_df = completed_df[['person','time','offer_id','reward']].reset_index(drop=True)
completed_df.head()

,person,time,offer_id,reward
0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,2906b810c7d4411798c6938adc9daaa5,2.0
1,fe97aa22dd3e48c8b143116a8403dd52,0,fafdcd668e3743c1bb461111dcafc2a4,2.0
2,629fc02d56414d91bca360decdfa9288,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0
3,676506bad68e4161b9bbaffeb039626b,0,ae264e3637204a6fb9bb56bc8210ddfd,10.0
4,8f7dd3b2afe14c078eb4f6e6fe4ba97d,0,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0


In [20]:
received_completed_df = pd.merge(received_df[['marketing_id','person','time','duration','offer_id']]
                            ,completed_df
                           ,on=['person','offer_id']
                           ,how='inner')
received_completed_df['In Scope'] = ((received_completed_df['time_y'] >= received_completed_df['time_x'])
                                     & (received_completed_df['time_y'] <= (received_completed_df['time_x'] 
                                                                              + received_completed_df['duration']
                                                                             )
                                       )
                                  )
received_completed_df.head()

,marketing_id,person,time_x,duration,offer_id,time_y,reward,In Scope
0,78afa995795e4d85b5d9ceeca43f5fef_0_9b98b8c7a33...,78afa995795e4d85b5d9ceeca43f5fef,0,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,132,5.0,False
1,389bc3fa690240e798340f5a15918d5c_0_f19421c1d4a...,389bc3fa690240e798340f5a15918d5c,0,5,f19421c1d4aa40978ebb69ca19b0e20d,60,5.0,False
2,389bc3fa690240e798340f5a15918d5c_0_f19421c1d4a...,389bc3fa690240e798340f5a15918d5c,0,5,f19421c1d4aa40978ebb69ca19b0e20d,600,5.0,False
3,389bc3fa690240e798340f5a15918d5c_504_f19421c1d...,389bc3fa690240e798340f5a15918d5c,504,5,f19421c1d4aa40978ebb69ca19b0e20d,60,5.0,False
4,389bc3fa690240e798340f5a15918d5c_504_f19421c1d...,389bc3fa690240e798340f5a15918d5c,504,5,f19421c1d4aa40978ebb69ca19b0e20d,600,5.0,False


In [46]:
completed_in_time = received_completed_df[received_completed_df['In Scope']]['marketing_id'].unique()
received_df['completed_in_time'] = received_df['marketing_id'].isin(completed_in_time)
received_df.head()

,person,time,offer_id,duration,marketing_id,viewed_in_time,completed_in_time
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,7,78afa995795e4d85b5d9ceeca43f5fef_0_9b98b8c7a33...,True,False
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,10,a03223e636434f42ac4c3df47e8bac43_0_0b1e1539f2c...,True,False
2,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5,7,e2127556f4f64592b11af22de27a7932_0_2906b810c7d...,False,False
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4,10,8ec6ce2a7e7949b1bf142def7d0e0586_0_fafdcd668e3...,False,False
4,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0,5,68617ca6246f4fbc85e91a2a49552598_0_4d5c57ea9a6...,False,False


In [24]:
received_df['completed_in_time'].value_counts()

False    72675
True      3602
Name: completed_in_time, dtype: int64

In [26]:
received_df['viewed_in_time'].value_counts()

False    58606
True     17671
Name: viewed_in_time, dtype: int64

In [37]:
#Looking at where the offer was completed but not viewed
received_df[(received_df['viewed_in_time'] == False) & (received_df['completed_in_time'] == True)].head()

,person,time,offer_id,duration,marketing_id,viewed_in_time,completed_in_time
126,2481f1fcfbcb4b288e5a03af02d95373,0,2906b810c7d4411798c6938adc9daaa5,7,2481f1fcfbcb4b288e5a03af02d95373_0_2906b810c7d...,False,True
195,d0be9ff460964c3398a33ad9b2829f3a,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,7,d0be9ff460964c3398a33ad9b2829f3a_0_9b98b8c7a33...,False,True
317,bb0f25e23a4c4de6a645527c275cd594,0,2298d6c36e964ae4a3e7e9706d1fb8c2,7,bb0f25e23a4c4de6a645527c275cd594_0_2298d6c36e9...,False,True
589,b860d355ef6e4c66b5d5a837c56ef32d,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,7,b860d355ef6e4c66b5d5a837c56ef32d_0_9b98b8c7a33...,False,True
600,99297ea01107436fa8c2e2bc86f55d89,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,7,99297ea01107436fa8c2e2bc86f55d89_0_9b98b8c7a33...,False,True


In [47]:
received_df['final_completed'] = received_df['completed_in_time'] & received_df['viewed_in_time']
received_df.head()

,person,time,offer_id,duration,marketing_id,viewed_in_time,completed_in_time,final_completed
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,7,78afa995795e4d85b5d9ceeca43f5fef_0_9b98b8c7a33...,True,False,False
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,10,a03223e636434f42ac4c3df47e8bac43_0_0b1e1539f2c...,True,False,False
2,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5,7,e2127556f4f64592b11af22de27a7932_0_2906b810c7d...,False,False,False
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4,10,8ec6ce2a7e7949b1bf142def7d0e0586_0_fafdcd668e3...,False,False,False
4,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0,5,68617ca6246f4fbc85e91a2a49552598_0_4d5c57ea9a6...,False,False,False


In [40]:
received_df['final_completed'].value_counts()

False    74615
True      1662
Name: final_completed, dtype: int64

In [48]:
received_df = received_df.merge(portfolio[['id','difficulty','channels','offer_type','reward']]
                                ,how='left'
                                , left_on='offer_id'
                                , right_on='id')
received_df.drop('id',axis=1,inplace=True)
received_df = received_df.merge(profile[['id','age','became_member_on','gender','income']]
                                ,how='left'
                                , left_on='person'
                                , right_on='id')
received_df.drop('id',axis=1,inplace=True)
received_df.head()

,person,time,offer_id,duration,marketing_id,viewed_in_time,completed_in_time,final_completed,difficulty,channels,offer_type,reward,age,became_member_on,gender,income
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,7,78afa995795e4d85b5d9ceeca43f5fef_0_9b98b8c7a33...,True,False,False,5,"[web, email, mobile]",bogo,5,75,20170509,F,100000.0
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,10,a03223e636434f42ac4c3df47e8bac43_0_0b1e1539f2c...,True,False,False,20,"[web, email]",discount,5,118,20170804,None,NaN
2,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5,7,e2127556f4f64592b11af22de27a7932_0_2906b810c7d...,False,False,False,10,"[web, email, mobile]",discount,2,68,20180426,M,70000.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4,10,8ec6ce2a7e7949b1bf142def7d0e0586_0_fafdcd668e3...,False,False,False,10,"[web, email, mobile, social]",discount,2,118,20170925,None,NaN
4,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0,5,68617ca6246f4fbc85e91a2a49552598_0_4d5c57ea9a6...,False,False,False,10,"[web, email, mobile, social]",bogo,10,118,20171002,None,NaN


In [49]:
received_df.to_pickle('received_df.pickle')

In [ ]:
received_df = pd.read_pickle('received_df.pickle')

### Getting data ready for machine learning model

In [63]:
received_df['channels'].head()

0            [web, email, mobile]
1                    [web, email]
2            [web, email, mobile]
3    [web, email, mobile, social]
4    [web, email, mobile, social]
Name: channels, dtype: object

In [62]:
received_df['channels'].head().str.split(',')

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: channels, dtype: float64

In [79]:
transcript[transcript['event'] == 'offer viewed'].head()

,event,person,time,value
12650,offer viewed,389bc3fa690240e798340f5a15918d5c,0,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'}
12651,offer viewed,d1ede868e29245ea91818a903fec04c6,0,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
12652,offer viewed,102e9454054946fda62242d2e176fdce,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
12653,offer viewed,02c083884c7d45b39cc68e1314fec56c,0,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'}
12655,offer viewed,be8a5d1981a2458d90b255ddc7e0d174,0,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}


In [15]:
transaction_df = transcript[transcript['event'] == 'transaction'].copy()
transaction_df.reset_index(inplace=True)
transaction_split = json_normalize(transaction_df['value'])
transaction_split.head()

,amount
0,0.83
1,34.56
2,13.23
3,19.51
4,18.97


In [19]:
transaction_df['amount'] = transaction_split['amount']
transaction_df.drop(['index','event','value'],axis=1,inplace=True)
transaction_df.head()

,person,time,amount
0,02c083884c7d45b39cc68e1314fec56c,0,0.83
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56
2,54890f68699049c2a04d415abc25e717,0,13.23
3,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51
4,fe97aa22dd3e48c8b143116a8403dd52,0,18.97


In [21]:
offer_received_df = transcript[transcript['event'] == 'offer received'].copy()
offer_received_df.reset_index(inplace=True)
received_split = json_normalize(offer_received_df['value'])
received_split.head()

,offer id
0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,2906b810c7d4411798c6938adc9daaa5
3,fafdcd668e3743c1bb461111dcafc2a4
4,4d5c57ea9a6940dd891ad53e9dbe8da0


In [25]:
offer_received_df['offer_id'] = received_split['offer id']
offer_received_df.drop(['index','event','value'],axis=1,inplace=True)
offer_received_df.head()

,person,time,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0


In [27]:
offer_viewed_df = transcript[transcript['event'] == 'offer viewed'].copy()
offer_viewed_df.reset_index(inplace=True)
viewed_split = json_normalize(offer_viewed_df['value'])
viewed_split.head()

,offer id
0,f19421c1d4aa40978ebb69ca19b0e20d
1,5a8bc65990b245e5a138643cd4eb9837
2,4d5c57ea9a6940dd891ad53e9dbe8da0
3,ae264e3637204a6fb9bb56bc8210ddfd
4,5a8bc65990b245e5a138643cd4eb9837


In [28]:
offer_viewed_df['offer_id'] = viewed_split['offer id']
offer_viewed_df.drop(['index','event','value'],axis=1,inplace=True)
offer_viewed_df.head()

,person,time,offer_id
0,389bc3fa690240e798340f5a15918d5c,0,f19421c1d4aa40978ebb69ca19b0e20d
1,d1ede868e29245ea91818a903fec04c6,0,5a8bc65990b245e5a138643cd4eb9837
2,102e9454054946fda62242d2e176fdce,0,4d5c57ea9a6940dd891ad53e9dbe8da0
3,02c083884c7d45b39cc68e1314fec56c,0,ae264e3637204a6fb9bb56bc8210ddfd
4,be8a5d1981a2458d90b255ddc7e0d174,0,5a8bc65990b245e5a138643cd4eb9837


In [29]:
offer_completed_df = transcript[transcript['event'] == 'offer completed'].copy()
offer_completed_df.reset_index(inplace=True)
completed_split = json_normalize(offer_completed_df['value'])
completed_split.head()

,offer_id,reward
0,2906b810c7d4411798c6938adc9daaa5,2
1,fafdcd668e3743c1bb461111dcafc2a4,2
2,9b98b8c7a33c4b65b9aebfe6a799e6d9,5
3,ae264e3637204a6fb9bb56bc8210ddfd,10
4,4d5c57ea9a6940dd891ad53e9dbe8da0,10


In [30]:
offer_completed_df['offer_id'] = completed_split['offer_id']
offer_completed_df['reward'] = completed_split['reward']
offer_completed_df.drop(['index','event','value'],axis=1,inplace=True)
offer_completed_df.head()

,person,time,offer_id,reward
0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,2906b810c7d4411798c6938adc9daaa5,2
1,fe97aa22dd3e48c8b143116a8403dd52,0,fafdcd668e3743c1bb461111dcafc2a4,2
2,629fc02d56414d91bca360decdfa9288,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5
3,676506bad68e4161b9bbaffeb039626b,0,ae264e3637204a6fb9bb56bc8210ddfd,10
4,8f7dd3b2afe14c078eb4f6e6fe4ba97d,0,4d5c57ea9a6940dd891ad53e9dbe8da0,10


In [29]:
#using json_normalize instead of a customer apply function for better performance
value_split = json_normalize(transcript['value'])
value_split.columns
#pd.concat([transcript, value_split], axis=1)

Index(['amount', 'offer id', 'offer_id', 'reward'], dtype='object')

In [44]:
#looking to see if multiple events are recorded in single values
value_split.head()

,amount,offer id,offer_id,reward
0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,NaN,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


In [49]:
nulls_per_row = value_split.isnull().sum(axis=1)

In [50]:
nulls_per_row.value_counts()

3    272955
2     33579
dtype: int64

In [72]:
#viewing some rows
test_twos = nulls_per_row[nulls_per_row == 2].head().index
value_split.loc[test_twos.tolist()]

,amount,offer id,offer_id,reward
12658,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,2.0
12672,NaN,NaN,fafdcd668e3743c1bb461111dcafc2a4,2.0
12679,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0
12692,NaN,NaN,ae264e3637204a6fb9bb56bc8210ddfd,10.0
12697,NaN,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0


In [73]:
value_split[~value_split['amount'].isnull()].head()

,amount,offer id,offer_id,reward
12654,0.83,NaN,NaN,NaN
12657,34.56,NaN,NaN,NaN
12659,13.23,NaN,NaN,NaN
12670,19.51,NaN,NaN,NaN
12671,18.97,NaN,NaN,NaN


In [78]:
transcript.loc[[0,1,12651,12652,12653,12654,12657,12672,12679]]

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
12651,offer viewed,d1ede868e29245ea91818a903fec04c6,0,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
12652,offer viewed,102e9454054946fda62242d2e176fdce,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
12653,offer viewed,02c083884c7d45b39cc68e1314fec56c,0,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'}
12654,transaction,02c083884c7d45b39cc68e1314fec56c,0,{'amount': 0.8300000000000001}
12657,transaction,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,{'amount': 34.56}
12672,offer completed,fe97aa22dd3e48c8b143116a8403dd52,0,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...
12679,offer completed,629fc02d56414d91bca360decdfa9288,0,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...


In [82]:
transcript[transcript['person'] == 'fe97aa22dd3e48c8b143116a8403dd52']

,event,person,time,value
107,offer received,fe97aa22dd3e48c8b143116a8403dd52,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
12671,transaction,fe97aa22dd3e48c8b143116a8403dd52,0,{'amount': 18.97}
12672,offer completed,fe97aa22dd3e48c8b143116a8403dd52,0,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...
15578,offer viewed,fe97aa22dd3e48c8b143116a8403dd52,6,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
24238,transaction,fe97aa22dd3e48c8b143116a8403dd52,30,{'amount': 13.9}
38230,transaction,fe97aa22dd3e48c8b143116a8403dd52,84,{'amount': 16.99}
39554,transaction,fe97aa22dd3e48c8b143116a8403dd52,90,{'amount': 13.96}
53283,offer received,fe97aa22dd3e48c8b143116a8403dd52,168,{'offer id': '3f207df678b143eea3cee63160fa8bed'}
79443,offer viewed,fe97aa22dd3e48c8b143116a8403dd52,198,{'offer id': '3f207df678b143eea3cee63160fa8bed'}
79444,transaction,fe97aa22dd3e48c8b143116a8403dd52,198,{'amount': 28.71}
